<a href="https://colab.research.google.com/github/tho071206-ther/TriTueNhanTao/blob/main/minimax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bai thuc hanh tuan 3

In [ ]:
import math
PLAYER = 'X' # Người chơi MAX (AI)
OPPONENT = 'O' # Người chơi MIN (Đối thủ)

def get_board_size(): #Hàm cho phép người dùng nhập kích thước N của bàn cờ N*N (N >= 3).

    while True:
        try:
            n = int(input("NHẬP: Kích thước bàn cờ N (N >= 3, ví dụ 3, 5, 10): "))
            if n >= 3:
                return n
            else:
                print("LỖI: Kích thước N phải lớn hơn hoặc bằng 3. Vui lòng nhập lại.")
        except ValueError:
            print("LỖI: Đầu vào không hợp lệ. Vui lòng nhập một số nguyên.")

def check_win(board, player):
    # Kiểm tra xem 'player' có đạt N quân liên tiếp hay không.
    # Sinh viên tự cài đặt logic kiểm tra thắng cho N*N/5-liên-tiếp ***
    # Đây là phần phức tạp nhất, cần kiểm tra 4 hướng (Ngang, Dọc, 2 Chéo).
    return False

def is_terminal(board):
    """
    Kiểm tra xem bàn cờ đã kết thúc (thắng, thua, hoặc hòa) hay chưa.
    """
    # 1. Kiểm tra thắng/thua
    if check_win(board, PLAYER) or check_win(board, OPPONENT):
        return True

    # 2. Kiểm tra hết ô trống (Hòa)
    for row in board:
        if ' ' in row:
            return False
    return True

def evaluate(board):
    """
    Hàm đánh giá Heuristic.
    Gán điểm số cho bàn cờ. Giá trị này càng lớn càng có lợi cho AI (MAX).
    """
    # *** CẦN TRIỂN KHAI PHỨC TẠP CHO CỜ CARO N*N ***
    # Ở đây dùng giá trị đơn giản cho mục đích demo logic Minimax.
    if check_win(board, PLAYER):
        return 1000000 # AI thắng: Điểm rất cao
    if check_win(board, OPPONENT):
        return -1000000 # Đối thủ thắng: Điểm rất thấp
    return 0 # Hòa hoặc thế cờ chưa rõ ràng

def get_available_moves(board):
    """
    Tìm tất cả các nước đi hợp lệ (ô trống).
    """
    moves = []
    N = len(board)
    for row in range(N):
        for col in range(N):
            if board[row][col] == ' ':
                moves.append((row, col))
    return moves

# =========================================================================================
# II. THUẬT TOÁN MINIMAX (PHIÊN BẢN CƠ BẢN)
# =========================================================================================

def minimax(board, depth, is_maximizing_player):
    """
    Hàm Minimax đệ quy.

    Tham số:
    - board: Trạng thái bàn cờ hiện tại.
    - depth: Độ sâu còn lại để tìm kiếm.
    - is_maximizing_player: True nếu đang ở lượt của AI (MAX), False nếu là lượt của Đối thủ (MIN).
    """

    # ĐIỀU KIỆN DỪNG: Đạt độ sâu giới hạn HOẶC Trạng thái kết thúc
    if depth == 0 or is_terminal(board):
        return evaluate(board)

    if is_maximizing_player:
        # LƯỢT CỦA MAX (AI) - Tìm giá trị lớn nhất
        max_eval = -math.inf # Khởi tạo bằng âm vô cùng

        for move in get_available_moves(board):
            row, col = move

            # 1. Thực hiện nước đi ảo
            board[row][col] = PLAYER

            # 2. Gọi đệ quy cho lượt của MIN
            eval = minimax(board, depth - 1, False)

            # 3. Hoàn trả trạng thái (Backtrack)
            board[row][col] = ' '

            # 4. Cập nhật giá trị tốt nhất (tối đa hóa)
            max_eval = max(max_eval, eval)

        return max_eval

    else:
        # LƯỢT CỦA MIN (Đối thủ) - Tìm giá trị nhỏ nhất
        min_eval = math.inf # Khởi tạo bằng dương vô cùng

        for move in get_available_moves(board):
            row, col = move

            # 1. Thực hiện nước đi ảo
            board[row][col] = OPPONENT

            # 2. Gọi đệ quy cho lượt của MAX
            eval = minimax(board, depth - 1, True)

            # 3. Hoàn trả trạng thái (Backtrack)
            board[row][col] = ' '

            # 4. Cập nhật giá trị tốt nhất (tối thiểu hóa)
            min_eval = min(min_eval, eval)

        return min_eval

# =========================================================================================
# III. HÀM TÌM NƯỚC ĐI TỐT NHẤT (ENTRY POINT)
# =========================================================================================

def find_best_move_minimax(board, depth):
    """
    Hàm gọi Minimax để tìm nước đi tối ưu nhất cho AI.
    """
    best_eval = -math.inf
    best_move = None

    for move in get_available_moves(board):
        row, col = move

        board[row][col] = PLAYER # Thử nước đi

        # Gọi Minimax. Lượt kế tiếp là lượt của Đối thủ (MIN).
        eval = minimax(board, depth - 1, False)

        board[row][col] = ' ' # Hoàn trả trạng thái

        # Nếu tìm thấy giá trị tốt hơn, cập nhật
        if eval > best_eval:
            best_eval = eval
            best_move = move

    return best_move, best_eval

# =========================================================================================
# IV. DEMO CHƯƠNG TRÌNH
# =========================================================================================

if __name__ == '__main__':
    N = get_board_size()
    board = [[' ' for _ in range(N)] for _ in range(N)]

    # Độ sâu tìm kiếm: Phải nhỏ nếu N lớn (ví dụ: N=10 chỉ nên dùng depth 3-4)
    search_depth = 4

    print(f"\n--- DEMO THUẬT TOÁN MINIMAX ({N}x{N}) ---")
    print(f"LƯU Ý: Minimax cơ bản RẤT CHẬM với N lớn.")

    # Giả lập một bước đi đầu tiên của AI
    print(f"\n[AI đang suy nghĩ với độ sâu = {search_depth}]")

    best_move, score = find_best_move_minimax(board, search_depth)

    if best_move:
        row, col = best_move
        board[row][col] = PLAYER
        print(f"-> Nước đi tối ưu (Minimax): ({row}, {col})")
        print(f"-> Giá trị thế cờ (Score): {score}")
    else:
        print("Không còn nước đi nào.")

hekkk
